In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense
import tensorflow.keras.utils as utils
import numpy as np
import math
import gym

In [ ]:
def create_model():
    model = Sequential()
    model.add(Input(shape = (4,)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    #model.add(Dense(8, activation='relu'))
    #model.add(Dense(4, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    return model

In [ ]:
def take_probabilistic_action(model, state):
    probabilities = model.predict(state.reshape(1, -1))
    action = np.random.choice(2, p=probabilities[0])
    return action

In [ ]:
def play_and_display(gym_env, model, duration = 1000):
    state = gym_env.reset()
    for _ in range(duration):
        gym_env.render()
        action = take_probabilistic_action(model, state)
        state, reward, done, info = gym_env.step(action)
    gym_env.close()

In [ ]:
play_and_display(gym.make('CartPole-v0'), create_model())

In [ ]:
### TODODODO
def compute_discounted_R(R, discount_rate=.99):
    """Returns discounted rewards
    Args:
        R (1-D array): a list of `reward` at each time step
        discount_rate (float): Will discount the future value by this rate
    Returns:
        discounted_r (1-D array): same shape as input `R`
            but the values are discounted
    Examples:
        >>> R = [1, 1, 1]
        >>> compute_discounted_R(R, .99) # before normalization
        [1 + 0.99 + 0.99**2, 1 + 0.99, 1]
    """
    discounted_r = np.zeros_like(R, dtype=np.float32)
    running_add = 0
    for t in reversed(range(len(R))):

        running_add = running_add * discount_rate + R[t]
        discounted_r[t] = running_add

    discounted_r -= discounted_r.mean() / discounted_r.std()

    return discounted_r

In [ ]:
def train_old(gym_env, model, duration=100, render=False):
    state = gym_env.reset()
    state_history = []
    action_history = []
    reward_history = []
    cummulative_reward = 0

    done = False
    # simulation
    #while not done:
    for _ in range(duration):
        if render:
            gym_env.render()
        action = take_probabilistic_action(model, state)
        action_history.append(action)
        state_history.append(state)
        state, reward, done, info = gym_env.step(action)
        cummulative_reward += reward
        reward_history.append(reward)
    gym_env.close()
    
    # prepare y - the "correct" action we'll reward
    for i in range(len(reward_history)):
        if reward_history[i] == 0.0:
            action_history[i] = int(not action_history[i])
    y = utils.to_categorical(action_history)
    
    # update weights
    model.fit(x=np.array(state_history), y=y)

In [ ]:
def get_loss(states, actions, rewards):
    pass

In [ ]:
#def custom_loss_function(y_true, y_pred):
def custom_loss_function(reward, action_prob):
    pass

In [ ]:
def train(gym_env, model, duration=100, render=False):
    state = gym_env.reset()
    state_history = []
    action_history = []
    reward_history = []
    cummulative_reward = 0

    done = False
    # simulation
    while not done:
    #for _ in range(duration):
        if render:
            gym_env.render()
        action = take_probabilistic_action(model, state)
        action_history.append(action)
        state_history.append(state)
        state, reward, done, info = gym_env.step(action)
        cummulative_reward += reward
        reward_history.append(reward)
    gym_env.close()
    
    # update weights
    #model.train_on_batch()
    model.fit(
        x = np.array(state_history),
        y = utils.to_categorical(action_history),
        sample_weight = compute_discounted_R(reward_history))

In [ ]:
env = gym.make('CartPole-v0')
model = create_model()

In [ ]:
for _ in range(2000):
    train(env, model)

In [ ]:
play_and_display(env, model)